In [1]:
import numpy as np
import astropy.units as u
import astropy.table
import plasmapy
import pathlib
import matplotlib.pyplot as plt

import fiasco
from fiasco.io import Parser

# MOXSI Line Catalogue

Start building a catalogue of the lines we want to target for diagnostics in our bandpass. We have a reduced table from the proposal but there are reasons to believe it is not complete and/or the wavelengths may not be as accurate as we would like. The goal here is to use the most recent version of CHIANTI to build a catalogue of the lines we believe will be important for both active regions and flares.

## Lines from Original Proposal

The following lines are those from Table 2 of the CubIXSS proposal. These are the lines I've been using to label my spectra thus far.

In [2]:
line_ids = [
    ('Fe XVIII',14.21*u.angstrom),  # also targeted by MaGIXS
    ('Fe XVII', 15.01*u.angstrom),  # also targeted by MaGIXS
    ('Fe XVII', 16.78*u.AA),
    ('Fe XVII', 17.05*u.AA),
    ('O VII', 21.60*u.angstrom),  # also targeted by MaGIXS
    ('O VII', 21.81*u.angstrom),
    ('O VII', 22.10*u.AA),
    ('O VIII', 18.97*u.angstrom),  # also targeted by MaGIXS
    ('Fe XXV', 1.86*u.AA),
    ('Ca XIX', 3.21*u.AA),
    ('Si XIII', 6.74*u.AA),
    ('Mg XI', 9.32*u.AA),
    ('Fe XVII', 11.25*u.AA),
    ('Fe XX', 12.83*u.AA),
    ('Ne IX', 13.45*u.AA),
    ('Fe XIX', 13.53*u.AA),
    ('C VI', 33.73*u.AA),
    ('C V', 40.27*u.AA),
    ('Si XII', 44.16*u.AA),
    ('Si XI', 49.18*u.AA),
]

In [3]:
element = [ion.split()[0] for ion,_ in line_ids]
ion = [ion.split()[1] for ion,_ in line_ids]
wavelengths = u.Quantity([line for _,line in line_ids])
proposal_table_2 = astropy.table.QTable({'element': element,
                                         'ionization stage': [plasmapy.utils.roman.from_roman(i) for i in ion],
                                         'ion name': [ion for ion,_ in line_ids],
                                         'wavelength': wavelengths})
proposal_table_2['energy'] = proposal_table_2['wavelength'].to('keV', equivalencies=u.spectral())

In [4]:
proposal_table_2.sort(keys='energy',reverse=True)

In [5]:
proposal_table_2

element,ionization stage,ion name,wavelength,energy
,,,Angstrom,keV
str2,int64,str8,float64,float64
Fe,25,Fe XXV,1.86,6.665817120064529
Ca,19,Ca XIX,3.21,3.8624360882616893
Si,13,Si XIII,6.74,1.8395281666646919
Mg,11,Mg XI,9.32,1.3303025582961399
Fe,17,Fe XVII,11.25,1.1020817638506688
Fe,20,Fe XX,12.83,0.9663616401652396
Ne,9,Ne IX,13.45,0.9218156017338308
Fe,19,Fe XIX,13.53,0.9163651029800463


## Line Lists from CHIANTI

I've run the `ch_ss` GUI from CHIANTI IDL and then exported the lines identified for the $1<\lambda<70$ Å range. I've built two tables, one for  the AR DEM and one for the extended flare DEM. I'm then combining these all into a single table by having a separate intensity column for each DEM. Then, I'm going to write this file back out that we can use as a reference when looking at model spectra.

In [6]:
def parse_chianti_line_file(filename):
    # Parse the emission line table
    line_table = astropy.table.QTable.read(filename, format='fits', hdu=1)
    line_table.convert_bytestring_to_unicode()
    # Parse the metadata associated with the emission calculation
    meta_table = astropy.table.Table.read(filename, format='fits', hdu=2)
    meta_table.convert_bytestring_to_unicode()
    # Add metadata to line table
    for col in meta_table.colnames:
        line_table.meta[col] = meta_table[col][0]
    # Rename some columns
    line_table.rename_columns(
        ('IZ', 'ION', 'SNOTE', 'LVL1', 'LVL2', 'TMAX', 'FLAG', 'INT', 'WVL', 'IDENT', 'IDENT_LATEX'),
        ('atomic number', 'ionization stage', 'ion name', 'lower level', 'upper level',
         'max temperature', 'only theoretical', 'intensity', 'wavelength', 'transition', 'transition (latex)'),
    )
    # Use unit information to add units to columns
    with u.add_enabled_aliases({'Angstroms': u.angstrom, 'photons': u.photon}):
        line_table['wavelength'].unit = line_table.meta['WVL_UNITS']
        line_table['intensity'].unit = line_table.meta['INT_UNITS']
    # Other fixes
    line_table['only theoretical'] = line_table['only theoretical'] == -1
    line_table['max temperature'] = (10**line_table['max temperature'] * u.K).to('MK')
    line_table['element'] = [i.split()[0] for i in line_table['ion name']]
    line_table['ion id'] = [f'{el.lower()}_{i}' for el,i in zip(line_table['element'], line_table['ionization stage'])]
    line_table['energy'] = line_table['wavelength'].to('keV', equivalencies=u.spectral())
    line_table['ion name'] = list(map(lambda x: x.strip(), line_table['ion name']))
    # Abundances
    ascii_dbase = '/Users/wtbarnes/ssw/packages/chianti/dbase/'
    abundance_file_coronal = 'sun_coronal_1992_feldman_ext.abund'
    abundance_file_photospheric = 'sun_photospheric_2015_scott.abund'
    coronal_abundances = fiasco.io.Parser(abundance_file_coronal, ascii_dbase_root=ascii_dbase).parse()
    photospheric_abundances = fiasco.io.Parser(abundance_file_photospheric, ascii_dbase_root=ascii_dbase).parse()
    line_table['abundance (coronal)'] =  0.0
    for row in coronal_abundances:
        line_table['abundance (coronal)'][line_table['element']==row['element']] = row['abundance']
    line_table['abundance (photospheric)'] = 0.0
    for row in photospheric_abundances:
        line_table['abundance (photospheric)'][line_table['element']==row['element']] = row['abundance']
    # Additional intensities
    line_table['intensity (coronal)'] = line_table['intensity'] * line_table['abundance (coronal)']
    line_table['intensity_scaled (coronal)'] = line_table['intensity (coronal)'] / line_table['intensity (coronal)'].max()
    line_table['intensity (photospheric)'] = line_table['intensity'] * line_table['abundance (photospheric)']
    line_table['intensity_scaled (photospheric)'] = line_table['intensity (photospheric)'] / line_table['intensity (photospheric)'].max()
    line_table.remove_column('intensity')
    return line_table

Parse the tables produced by CHIANTI. This parsing cleans up the column names and adds some additional columns, including the intensities scaled by the abundances.

In [7]:
ar_line_table = parse_chianti_line_file('../sandbox/data/moxsi_ar_lines.fits')
flare_ext_line_table = parse_chianti_line_file('../sandbox/data/moxsi_flare_ext_lines.fits')

Now, we combine these two separate tables into one, labeling the non-unique column names according to the DEM used.

In [8]:
unique_cols = ['max temperature',
               'only theoretical',
               'intensity (coronal)',
               'intensity (photospheric)',
               'intensity_scaled (coronal)',
               'intensity_scaled (photospheric)']
combined_line_list = astropy.table.join(
    flare_ext_line_table, ar_line_table,
    keys=[c for c in flare_ext_line_table.colnames if c not in unique_cols],
    table_names=['flare_ext', 'active_region'],
    join_type='outer'
)

This yields a really large line list so we want to pare it down a bit. We apply the condition that the active region or flare intensity must be above a given threshold.

In [9]:
threshold = 0.01
is_visible = np.logical_or(combined_line_list['intensity_scaled (coronal)_active_region']>threshold,
                           combined_line_list['intensity_scaled (coronal)_flare_ext']>threshold)

In [10]:
reduced_line_list = combined_line_list[np.where(is_visible)]

In [11]:
reduced_line_list

atomic number,ionization stage,transition,transition (latex),ion name,lower level,upper level,max temperature_flare_ext,wavelength,only theoretical_flare_ext,element,ion id,energy,abundance (coronal),abundance (photospheric),intensity (coronal)_flare_ext,intensity_scaled (coronal)_flare_ext,intensity (photospheric)_flare_ext,intensity_scaled (photospheric)_flare_ext,max temperature_active_region,only theoretical_active_region,intensity (coronal)_active_region,intensity_scaled (coronal)_active_region,intensity (photospheric)_active_region,intensity_scaled (photospheric)_active_region
,,,,,,,MK,Angstrom,,,,keV,,,ph / (cm2 s sr),,ph / (cm2 s sr),,MK,,ph / (cm2 s sr),,ph / (cm2 s sr),
int16,int16,str48,str93,str9,int16,int16,float32,float64,bool,str2,str5,float64,float64,float64,float64,float64,float64,float64,float32,bool,float64,float64,float64,float64
6,5,1s2 1S0 - 1s.2p 1P1,1s$^{2}$ $^1$S$_{0}$ - 1s 2p $^1$P$_{1}$,C V,1,7,1.0,40.267398834228516,False,C,c_5,0.3079021789900417,0.0003890451449942805,0.0002691534803926914,6533714714604.047,0.001743170922485647,4520226194711.858,0.004636371752013014,1.1220190525054932,False,329912553647.37476,0.14822303764598255,228243722308.20892,0.23036640952282866
6,5,1s2 1S0 - 1s.2p 3P1,1s$^{2}$ $^1$S$_{0}$ - 1s 2p $^3$P$_{1}$,C V,1,4,0.8912505507469177,40.730201721191406,False,C,c_5,0.3044035953514388,0.0003890451449942805,0.0002691534803926914,817185380850.2845,0.00021802203744748316,565354155454.413,0.0005798807235130193,1.0,False,47490837462.94683,0.021336672737326194,32855632191.743896,0.033161192536082684
6,5,1s2 1S0 - 1s.2s 3S1,1s$^{2}$ $^1$S$_{0}$ - 1s 2s $^3$S$_{1}$,C V,1,2,1.0,41.471500396728516,False,C,c_5,0.298962413337186,0.0003890451449942805,0.0002691534803926914,4448601168028.341,0.001186870339549656,3077680065308.1606,0.003156759927903988,1.1220190525054932,False,250709904086.47617,0.11263888912620688,173448876363.1402,0.1750619665656783
6,5,1s2 1S0 - 1s.3p 1P1,1s$^{2}$ $^1$S$_{0}$ - 1s 3p $^1$P$_{1}$,C V,1,17,1.1220190525054932,34.972801208496094,False,C,c_5,0.35451606433824995,0.0003890451449942805,0.0002691534803926914,970088269583.565,0.00025881596268700365,671137109423.0685,0.0006883817317586987,1.1220190525054932,False,44996472231.79761,0.020216004889219277,31129953072.051495,0.031419464444848534
6,6,1s 2S1/2 - 2p 2P1/2,1s $^2$S$_{1/2}$ - 2p $^2$P$_{1/2}$,C VI,1,3,11.220189094543457,33.739601135253906,False,C,c_6,0.3674738119640999,0.0003890451449942805,0.0002691534803926914,157924415453753.56,0.04213364999763524,109257001675175.8,0.11206432033623354,1.5848925113677979,False,523641758655.5146,0.23526165115003184,362271586304.34393,0.3656407447490605
6,6,1s 2S1/2 - 2p 2P3/2,1s $^2$S$_{1/2}$ - 2p $^2$P$_{3/2}$,C VI,1,4,11.220189094543457,33.73419952392578,False,C,c_6,0.3675326528654258,0.0003890451449942805,0.0002691534803926914,289421291045608.6,0.07721653009600904,200230612788695.44,0.20537546508354335,1.5848925113677979,False,855361668545.9404,0.384296697400997,591765693636.9792,0.5972691679898798
6,6,1s 2S1/2 - 3p 2P1/2,1s $^2$S$_{1/2}$ - 3p $^2$P$_{1/2}$,C VI,1,6,11.220189094543457,28.466299057006836,False,C,c_6,0.4355473052008219,0.0003890451449942805,0.0002691534803926914,20029673823547.336,0.00534384290119645,13857148688534.355,0.014213203051238077,1.5848925113677979,False,44602733254.508934,0.020039105930364474,30857552253.10524,0.031144530273704105
6,6,1s 2S1/2 - 3p 2P3/2,1s $^2$S$_{1/2}$ - 3p $^2$P$_{3/2}$,C VI,1,7,11.220189094543457,28.465200424194336,False,C,c_6,0.4355641154306379,0.0003890451449942805,0.0002691534803926914,40066139099582.625,0.010689497736800134,27718995914237.4,0.028431225366837343,1.5848925113677979,False,89273505126.47072,0.04010878023542279,61762175728.982864,0.062336569536800175


The resulting line list is greatly reduced compared to the original. Let's write our complete line list to an ASDF file which makes it really easy to load the whole table in.

In [12]:
reduced_line_list.write('../data/moxsi-line-list.asdf')

We can inspect our line list a bit further in terms of the two cases we're interested in, looking at the top AR and flare lines.

In [13]:
reduced_ar_line_list = reduced_line_list[reduced_line_list['intensity_scaled (coronal)_active_region']>threshold]
reduced_ar_line_list[
    reduced_ar_line_list.argsort(keys='intensity (coronal)_active_region', reverse=True)][
    ['ion name', 'wavelength', 'intensity_scaled (coronal)_active_region']
].pprint_all(show_name=False)

              Angstrom                          
-------- ------------------ --------------------
 Fe XVII 15.012999534606934                  1.0
   Fe XV  69.68199920654297   0.9640567860338324
 Fe XVII 17.051000595092773   0.9589103583046258
 Fe XVII  17.09600067138672   0.8640182350231367
 Fe XVII  16.77560043334961   0.7191395687147936
   O VII 21.601499557495117   0.7054992491378178
  Fe XVI  63.71099853515625    0.621815642790708
  O VIII 18.967100143432617   0.6143438040118606
   O VII 22.097700119018555   0.5105757744779554
  Fe XVI  66.35700225830078  0.43763281689012395
    C VI  33.73419952392578    0.384296697400997
   Si XI 49.222999572753906  0.36815618070031947
    Si X  50.52399826049805  0.36536935887445027
    Mg X  63.29499816894531   0.3529053479360131
    Si X 50.691001892089844  0.30915719392100166
  Si XII 44.160301208496094  0.30806816974214357
   Fe XV 59.404998779296875  0.30740895937047125
  O VIII  18.97249984741211  0.30720894659391895
  Fe XVI   66.249000

In [14]:
reduced_flare_line_list = reduced_line_list[reduced_line_list['intensity_scaled (coronal)_flare_ext']>threshold]
reduced_flare_line_list[
    reduced_flare_line_list.argsort(keys='intensity (coronal)_flare_ext', reverse=True)][
    ['ion name', 'wavelength', 'intensity_scaled (coronal)_flare_ext']
].pprint_all(show_name=False)

              Angstrom                          
-------- ------------------ --------------------
 Fe XVII 15.012999534606934                  1.0
 Fe XVII 17.051000595092773   0.7990830377280506
  Fe XXI 12.281999588012695   0.7525288356799003
 Fe XVII  17.09600067138672   0.7228795890158841
 Fe XVII  16.77560043334961   0.5949798048043987
Fe XXIII 11.737000465393066  0.49179069142967213
Fe XVIII 14.203900337219238   0.4896022804982188
  O VIII 18.967100143432617   0.4122510037782132
 Fe XXII 11.767499923706055   0.4118199400496306
 Fe XXIV 11.170900344848633  0.37157515994978524
Fe XVIII  16.07200050354004  0.33495904420023276
  Fe XIX 13.524900436401367   0.3077585024598136
 Fe XXIV 10.619000434875488    0.305398657251735
   Fe XX  12.82699966430664  0.29560189950656485
 Fe XVII  15.26200008392334  0.28855152685279967
Fe XXIII  12.16100025177002   0.2815051348779527
Fe XVIII 14.208800315856934  0.27566060993420255
  Mg XII   8.41919994354248  0.27343705400135815
   Fe XX 12.84539985

Finally, let's make sure that our new line list at least includes all of those lines from the proposal table with approximately the same wavelength.

In [15]:
matching_rows = [] 
for row in proposal_table_2:
    is_same_ion = np.logical_and(row['element'] == reduced_line_list['element'],
                                 row['ionization stage'] == reduced_line_list['ionization stage'])
    same_ion = reduced_line_list[np.where(is_same_ion)]
    i_wave = np.argmin(np.fabs(same_ion['wavelength'] - row['wavelength']))
    matching_rows.append(same_ion[i_wave])
proposal_table_2_from_line_list = astropy.table.vstack(matching_rows)

In [16]:
proposal_table_2[['element', 'ionization stage', 'ion name', 'wavelength']]

element,ionization stage,ion name,wavelength
,,,Angstrom
str2,int64,str8,float64
Fe,25,Fe XXV,1.86
Ca,19,Ca XIX,3.21
Si,13,Si XIII,6.74
Mg,11,Mg XI,9.32
Fe,17,Fe XVII,11.25
Fe,20,Fe XX,12.83
Ne,9,Ne IX,13.45
Fe,19,Fe XIX,13.53


In [17]:
proposal_table_2_from_line_list[['element', 'ionization stage', 'ion name', 'wavelength',
                                 'intensity_scaled (coronal)_active_region',
                                 'intensity_scaled (coronal)_flare_ext']]

element,ionization stage,ion name,wavelength,intensity_scaled (coronal)_active_region,intensity_scaled (coronal)_flare_ext
,,,Angstrom,,
str2,int16,str9,float64,float64,float64
Fe,25,Fe XXV,1.8595000505447388,3.756333185942363e-15,0.01894606208066694
Ca,19,Ca XIX,3.177299976348877,1.2336774759948343e-06,0.014185118905457011
Si,13,Si XIII,6.740300178527832,0.007675330348682831,0.1106974765714088
Mg,11,Mg XI,9.314299583435059,0.034417403994498975,0.06983122883637835
Fe,17,Fe XVII,11.25,0.0340219437587159,0.04458767268142438
Fe,20,Fe XX,12.82699966430664,0.0016536551510616972,0.29560189950656485
Ne,9,Ne IX,13.44729995727539,0.09882608471995388,0.0352391004263809
Fe,19,Fe XIX,13.524900436401367,0.011105918746710824,0.3077585024598136
